In [19]:
from langgraph.graph import StateGraph, START, END 
from typing_extensions import TypedDict

In [ ]:
class PrivateState(TypedDict):
    a: int
    b: int
    
class InputState(TypedDict):
    hello: str
    
class OutputState(TypedDict):
    bye: str
    
class MegaPrivate(TypedDict):
    secret: bool
    
graph_builder = StateGraph(PrivateState, input_schema=InputState, output_schema=OutputState)

In [ ]:
def node_one(state: InputState) -> InputState:
    """Receive InputState"""
    print("node one -> ", state)
    return {"hello": "world"}
    
def node_two(state: PrivateState) -> PrivateState:
    print("node two -> ", state)
    return {"a": 1 }
    
def node_three(state: PrivateState) -> PrivateState:
    print("node three -> ", state)
    return {"b": 1}

def node_four(state: PrivateState) -> OutputState:
    """Receive PrivateState"""
    print("node four -> ", state)
    return {"bye": "world"}

def node_five(state: OutputState) -> MegaPrivate:
    print("node five -> ", state)
    return {"secret": True}

def node_six(state: MegaPrivate) -> OutputState:
    print("node six -> ", state)
    

In [32]:
graph_builder.add_node("node_one", node_one)
graph_builder.add_node("node_two", node_two)
graph_builder.add_node("node_three", node_three)
graph_builder.add_node("node_four", node_four)

graph_builder.add_edge(START, "node_one")
graph_builder.add_edge("node_one", "node_two")
graph_builder.add_edge("node_two", "node_three")
graph_builder.add_edge("node_three", "node_four")
graph_builder.add_edge("node_four", END)

In [35]:
graph = graph_builder.compile()
print(graph.get_graph().draw_ascii())

+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
 +----------+  
 | node_one |  
 +----------+  
       *       
       *       
       *       
 +----------+  
 | node_two |  
 +----------+  
       *       
       *       
       *       
+------------+ 
| node_three | 
+------------+ 
       *       
       *       
       *       
+-----------+  
| node_four |  
+-----------+  
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


In [36]:
result = graph.invoke({"hello": "world"})
print("Result = ", result)

node one ->  {'hello': 'world'}
node two ->  {}
node three ->  {'a': 1}
node four ->  {'a': 1, 'b': 1}
Result =  {'bye': 'world'}
